In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = '/content/drive/MyDrive/news_summarizer_model'
base_model = 't5-small'   # 🔁 Replace with your actual base model name!

# Load tokenizer from the base model
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load your fine-tuned model weights from Drive
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

print("✅ Model and tokenizer loaded successfully!")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

✅ Model and tokenizer loaded successfully!


In [ ]:
news_article = """
India, officially the Republic of India,[j][20] is a country in South Asia. It is the seventh-largest country by area; the most populous country since 2023;[21] and, since its independence in 1947, the world's most populous democracy.[22][23][24] Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west;[k] China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is near Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime border with Myanmar, Thailand, and Indonesia.
Modern humans arrived on the Indian subcontinent from Africa no later than 55,000 years ago.[26][27][28] Their long occupation, predominantly in isolation as hunter-gatherers, has made the region highly diverse.[29] Settled life emerged on the subcontinent in the western margins of the Indus river basin 9,000 years ago, evolving gradually into the Indus Valley Civilisation of the third millennium BCE.[30] By 1200 BCE, an archaic form of Sanskrit, an Indo-European language, had diffused into India from the northwest.[31][32] Its hymns recorded the early dawnings of Hinduism in India.[33] India's pre-existing Dravidian languages were supplanted in the northern regions.[34] By 400 BCE, caste had emerged within Hinduism,[35] and Buddhism and Jainism had arisen, proclaiming social orders unlinked to heredity.[36] Early political consolidations gave rise to the loose-knit Maurya and Gupta Empires.[37] Widespread creativity suffused this era,[38] but the status of women declined,[39] and untouchability became an organised belief.[l][40] In South India, the Middle kingdoms exported Dravidian language scripts and religious cultures to the kingdoms of Southeast Asia.[41]
In the 1st millennium, Christianity, Islam, Judaism, and Zoroastrianism became established on India's southern and western coasts.[42] In the early centuries of the 2nd millennium Muslim armies from Central Asia intermittently overran India's northern plains.[43] The resulting Delhi Sultanate drew northern India into the cosmopolitan networks of medieval Islam.[44] In south India, the Vijayanagara Empire created a long-lasting composite Hindu culture.[45] In the Punjab, Sikhism emerged, rejecting institutionalised religion.[46] The Mughal Empire ushered in two centuries of economic expansion and relative peace,[47] and left a a rich architectural legacy.[48][49] Gradually expanding rule of the British East India Company turned India into a colonial economy but consolidated its sovereignty.[50] British Crown rule began in 1858. The rights promised to Indians were granted slowly,[51][52] but technological changes were introduced, and modern ideas of education and the public life took root.[53] A nationalist movement emerged in India, the first in the non-European British Empire and an influence on other nationalist movements.[54][55] Noted for nonviolent resistance after 1920,[56] it became the primary factor in ending British rule.[57] In 1947, the British Indian Empire was partitioned into two independent dominions,[58][59][60][61] a Hindu-majority dominion of India and a Muslim-majority dominion of Pakistan. A large-scale loss of life and an unprecedented migration accompanied the partition.[62]
India has been a federal republic since 1950, governed through a democratic parliamentary system. It is a pluralistic, multilingual and multi-ethnic society. India's population grew from 361 million in 1951 to over 1.4 billion in 2023.[63] During this time, its nominal per capita income increased from US$64 annually to US$2,601, and its literacy rate from 16.6% to 74%. A comparatively destitute country in 1951,[64] India has become a fast-growing major economy and a hub for information technology services, with an expanding middle class.[65] India has reduced its poverty rate, though at the cost of increasing economic inequality.[66] It is a nuclear-weapon state that ranks high in military expenditure. It has disputes over Kashmir with its neighbours, Pakistan and China, unresolved since the mid-20th century.[67] Among the socio-economic challenges India faces are gender inequality, child malnutrition,[68] and rising levels of air pollution.[69] India's land is megadiverse with four biodiversity hotspots.[70] India's wildlife, which has traditionally been viewed with tolerance in its culture,[71] is supported in protected habitats.
"""

# 💡 New Feature: Specify the target language for summarization.
# The model will attempt to summarize the article in this language.
# Examples: 'English', 'French', 'German', 'Spanish', etc.
target_language = 'English' # 🔁 Change this to your desired language

In [ ]:
!pip install deep-translator



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.5 MB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator

def auto_translate(summary):
    languages = {
        "hindi": "hi",
        "tamil": "ta",
        "telugu": "te",
        "marathi": "mr",
        "french": "fr",
        "spanish": "es",
        "german": "de",
        "kannada": "kn",
        "bengali": "bn",
    }

    translated = {}
    for lang, code in languages.items():
        translated[lang] = GoogleTranslator(source='auto', target=code).translate(summary)
    return translated


In [ ]:
def generate_summary(news_article, target_language):
    # Prepare multilingual input
    inputs_text = f"summarize {target_language}: {news_article}"

    inputs = tokenizer(inputs_text, return_tensors="pt", truncation=True, max_length=1024)

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=120,
        min_length=30,
        num_beams=4,
        length_penalty=2.0,
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
# Your news article (can come from scraping / API)
news_article = """
Apple announced a new AI-powered chip improving battery life and processing speed significantly.
"""

# Your model's target language for first summary
target_language = "english"   # you can change to Hindi, Tamil, etc.

# 1️⃣ Generate summary using your model
summary = generate_summary(news_article, target_language)
print("📰 MODEL SUMMARY:\n", summary)

# 2️⃣ Auto-translate summary into multiple languages
translations = auto_translate(summary)

# 3️⃣ Print translations cleanly
print("\n🌍 MULTI-LANGUAGE SUMMARIES")
for lang, translated_text in translations.items():
    print(f"\n--- {lang.upper()} ---")
    print(translated_text)


📰 MODEL SUMMARY:
 Apple announced a new AI-powered chip improving battery life and processing speed significantly. The AI-powered chip improves battery life and processing speed significantly.

🌍 MULTI-LANGUAGE SUMMARIES

--- HINDI ---
Apple ने एक नई AI-संचालित चिप की घोषणा की है जो बैटरी जीवन और प्रसंस्करण गति में उल्लेखनीय सुधार लाती है। एआई-संचालित चिप बैटरी जीवन और प्रसंस्करण गति में काफी सुधार करती है।

--- TAMIL ---
ஆப்பிள் புதிய AI-இயங்கும் சிப்பை அறிவித்தது, இது பேட்டரி ஆயுள் மற்றும் செயலாக்க வேகத்தை கணிசமாக மேம்படுத்துகிறது. AI-இயங்கும் சிப் பேட்டரி ஆயுள் மற்றும் செயலாக்க வேகத்தை கணிசமாக மேம்படுத்துகிறது.

--- TELUGU ---
యాపిల్ బ్యాటరీ లైఫ్ మరియు ప్రాసెసింగ్ వేగాన్ని గణనీయంగా మెరుగుపరిచే కొత్త AI-ఆధారిత చిప్‌ను ప్రకటించింది. AI-ఆధారిత చిప్ బ్యాటరీ జీవితాన్ని మరియు ప్రాసెసింగ్ వేగాన్ని గణనీయంగా మెరుగుపరుస్తుంది.

--- MARATHI ---
ऍपलने बॅटरीचे आयुष्य आणि प्रक्रियेची गती लक्षणीयरीत्या सुधारणारी नवीन एआय-चालित चिप जाहीर केली. AI-चालित चिप बॅटरीचे आयुष्य आणि प्रक्रिया गती लक्षणीयरी

In [ ]:
!pip install flask pyngrok

from flask import Flask, request, jsonify
from pyngrok import ngrok

# Set your ngrok authtoken here. Get it from https://dashboard.ngrok.com/get-started/your-authtoken
# 🔁 Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual ngrok authtoken!
ngrok.set_auth_token("31eePaiX63qaAwEl4f1YW6LZ31k_3u7wGCSUWzA2rPTFYpFRf")

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url)

app = Flask(__name__)

def summarize_news(text):
    # Your ML summarization code here
    # Example:
    summary = model.generate_summary(text)
    return summary

@app.route("/summarize", methods=["POST"])
def summarize():
    data = request.json["text"]
    result = summarize_news(data)
    return jsonify({"summary": result})

app.run(port=5000)

ERROR: Operation cancelled by user
Public URL: NgrokTunnel: "https://5d02ce7ff414.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

app = Flask(__name__)

model_path = "/content/drive/MyDrive/news_summarizer_model"
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

def generate_summary(text, target_language="english"):
    inputs_text = f"summarize {target_language}: {text}"
    inputs = tokenizer(inputs_text, return_tensors="pt", truncation=True, max_length=1024)

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=120,
        min_length=30,
        num_beams=4,
        length_penalty=2.0,
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

@app.route("/summarize", methods=["POST"])
def summarize():
    data = request.json.get("input", "")
    if not data:
        return jsonify({"summary": "No input received"}), 400
    summary = generate_summary(data)
    return jsonify({"summary": summary})

# Start ngrok tunnel
ngrok.set_auth_token("31eePaiX63qaAwEl4f1YW6LZ31k_3u7wGCSUWzA2rPTFYpFRf")
public_url = ngrok.connect(5000)
print("PUBLIC URL:", public_url)

app.run(port=5000)


PUBLIC URL: NgrokTunnel: "https://69837ac50142.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
